In [ ]:
import numpy as np
import pandas as pd
import math
import json

In [ ]:
ratingsDataframe = pd.read_csv("./ratings_small.csv")
ratingsDataframe.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
moviesDataframe = pd.read_csv("./links_small.csv")
moviesDataframe.dropna()
moviesDataframe['imdbId'] = moviesDataframe['imdbId'].astype(str)
moviesDataframe['imdbId'] = moviesDataframe['imdbId'].transform(lambda x: "tt"+ "0"*(7 - len(x)) + x)
moviesDataframe.head(5)

,movieId,imdbId,tmdbId
0,1,tt0114709,862.0
1,2,tt0113497,8844.0
2,3,tt0113228,15602.0
3,4,tt0114885,31357.0
4,5,tt0113041,11862.0


In [ ]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

In [ ]:
w = {}

In [ ]:
userRange = list(set(ratingsDataframe['userId']))

In [ ]:
def saveWeights(weights):
  w_file = open("weights.json", "w")
  json.dump(weights, w_file)
  w_file.close()

In [ ]:
def calculateSimilarity():
  le = len(userRange)
  # Cada usuario u de U
  for u in userRange:
    print(f"{u} -> {u/le}")
    # Pega seus Ratings e calc media
    uMovieIdRated = ratingsDataframe.query(f"userId == {u}")
    uMeanRating = uMovieIdRated['rating'].mean()
    restOfUsers = userRange[userRange.index(u) + 1:]
    # Cada usuario v de U - {u}
    # for v in userRange:
    for v in restOfUsers:
      if u == v: continue
      vMovieIdRated = ratingsDataframe.query(f"userId == {v}")

      # Pega a intersecçao i de ratings entre u e v
      intersect = intersection(list(uMovieIdRated['movieId']), list(vMovieIdRated['movieId']))
      # print(f"{v}", intersect)
      #Se nao tiverem instersecçao, provavelmente, nao sao proximos
      if len(intersect) == 0:
        # salvando como 0, pois um usuario neutro pode valer mais doq um oposto, caso não tenha usuario parecido <- pode ser uma politica de novo usuario
        w[f"{u},{v}"] = 0
        continue 

      # Calc media ratings v
      vMeanRating = vMovieIdRated['rating'].mean()
      # Calc somatorio####
      w_uv_numerador = 0
      u_i_denominador = 0
      v_i_denominador = 0
      for i in intersect:
        prod_rating_u_i = uMovieIdRated.query(f"movieId == {i}").iat[0,2]
        prod_rating_v_i = vMovieIdRated.query(f"movieId == {i}").iat[0,2]
        
        w_uv_numerador += (prod_rating_u_i - uMeanRating) * (prod_rating_v_i - vMeanRating)
        u_i_denominador += (prod_rating_u_i - uMeanRating)**2
        v_i_denominador += (prod_rating_v_i - vMeanRating)**2
      
      u_i_denominador = math.sqrt(u_i_denominador)
      v_i_denominador = math.sqrt(v_i_denominador)

      w_uv = w_uv_numerador/(u_i_denominador*v_i_denominador)

      w[f"{u},{v}"] = w_uv
      w[f"{v},{u}"] = w_uv
    
  saveWeights(w)



In [ ]:
calculateSimilarity()

Recomendação


In [ ]:
weightFile = open("./weights.json","r")
weightsStr = weightFile.read()
weights = json.loads(weightsStr)
weightFile.close()

In [ ]:
def getTopKUsers(U, K):
  df = pd.DataFrame(list(weights.items()),columns = ['id','val']) 
  dfU = df[df.id.str.startswith(f"{U},")]
  sortedFd = dfU.sort_values(by=['val'],ascending=False)
  sortedFd['val'].replace(np.nan, 0, inplace = True)
  topK = sortedFd.head(K)
  return topK

In [ ]:
def getUser(u):
  return ratingsDataframe[ratingsDataframe.userId == u]

In [ ]:
def getUnwatchedMovieId(U):
  uDf = getUser(U)  
  uListReviewedMovies = uDf.movieId
  uNotWatched = moviesDataframe[~moviesDataframe.isin(uListReviewedMovies)]
  # print(uNotWatched.isinf().sum())
  # uNotWatched.replace([np.inf, -np.inf], np.nan, inplace=True)
  # uNotWatched.dropna()
  return uNotWatched.movieId.unique()

In [ ]:
def recomend(U, topK):
  predict_u_i = {}
  uUnwatched = getUnwatchedMovieId(U)  
  # [print(type(uw)) for uw in uUnwatched]
  for movie in uUnwatched:
    numerador = 0
    denominador = 0
    for index, row in topK.iterrows():
      vId = int(row['id'].split(",")[1])
      v = getUser(vId)
      if movie in v.movieId.unique():
        r = v[v.movieId == movie]
        numerador += r.rating.iat[0] * weights[f"{U},{vId}"]
        denominador += abs(weights[f"{U},{vId}"])
      else: continue
    predict_u_i[f"{U},{movie}"] = numerador/denominador if denominador != 0 else 0

  return predict_u_i

In [ ]:
K = 5
U = 4
topK = getTopKUsers(U, K)

In [ ]:
recomendations = recomend(U, topK)

In [ ]:
sortedRec = sorted(recomendations.items(), key= lambda x:x[1], reverse=True)

In [ ]:
moviesNameDataframe = pd.read_csv("./movies_metadata.csv")
# moviesNameDataframe.head(5)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
numReco = 10
rui = sortedRec[0:numReco]
moviesNames = []
for m in rui:
  # v = moviesDataframe[moviesDataframe.movieId == int(m[0].split(",")[1].split(".")[0])]
  v = moviesDataframe[moviesDataframe.movieId == int(float(m[0].split(",")[1]))]
  a = moviesNameDataframe[moviesNameDataframe.imdb_id == v.imdbId.iat[0]]
  moviesNames.append(a.original_title.iat[0])

print(f"As recomendações do usuário {U} :")

for movie in moviesNames:
  print(f"- {movie}")


As recomendações do usuário 4 :
- Les misérables
- Good Will Hunting
- The Goonies
- Charlotte's Web
- The Sixth Sense
- American Beauty
- Chicago
- 올드보이
- Children of Men
- Hairspray
